In [1]:
from elasticsearch import Elasticsearch, helpers
import pandas as pd
from tqdm import tqdm
import time
import json
import re

from preprocess import preprocess

class elastic:
    def __init__(self, INDEX_NAME, context_path="../data/wikipedia_documents.json"):
        self.index_name = INDEX_NAME
        try:
            self.es.transport.close()
        except:
            pass
        self.context_path = context_path
        self.es = Elasticsearch(timeout=100, max_retries=10, retry_on_timeout=True)
        self.index_setting = {
            "settings": {
                "index": {
                    "analysis": {
                        "analyzer": {
                            "korean": {
                                "type": "custom",
                                "tokenizer": "nori_tokenizer",
                                "filter": ["shingle"],
                            }
                        }
                    }
                }
            },
            "mappings": {
                "properties": {
                    "text": {
                        "type": "text",
                        "analyzer": "korean",
                        "search_analyzer": "korean",
                    },
                    "title": {
                        "type": "text",
                        "analyzer": "korean",
                        "search_analyzer": "korean",
                    },
                }
            },
        }

    def build_elatic(self):
        with open(self.context_path) as file:
            json_data = json.load(file)
        docs = []
        for i, j in json_data.items():
            docs.append(
                {
                    "_index": "wikipedia",
                    "_source": {"text": j["text"], "title": j["title"]},
                }
            )

        if self.es.indices.exists(self.index_name):
            pass
        else:
            self.es.indices.create(index=self.index_name, body=self.index_setting)
            helpers.bulk(self.es, docs)

    def retrieve(self, query_or_dataset, topk):
        datas = []
        for i in tqdm(range(len(query_or_dataset))):
            cp = {i: v for i, v in query_or_dataset[i].items()}
            if (
                "context" in query_or_dataset[i].keys()
                and "answers" in query_or_dataset[i].keys()
            ):
                cp["original_context"] = query_or_dataset[i]["context"]

            query = query.replace("/", "")
            query = query.replace("~", " ")
            res = self.es.search(index=self.index_name, q=query, size=topk+1)
            x = res["hits"]["hits"]
            context = []
            for docu in x:
                context.append(docu["_source"]["text"])
            cp["context"] = "///".join(context)
            datas.append(cp)

        return pd.DataFrame(datas)

    def retrieve_false(self, query_or_dataset, topk):
        datas = []
        for i in tqdm(range(len(query_or_dataset))):
            cp = {i: v for i, v in query_or_dataset[i].items()}
            if (
                "context" in query_or_dataset[i].keys()
                and "answers" in query_or_dataset[i].keys()
            ):
                cp["original_context"] = preprocess(query_or_dataset[i]["context"])

            query = preprocess(query_or_dataset[i]["question"])
            query = query.replace("/", "")
            query = query.replace("~", " ")
            res = self.es.search(index=self.index_name, q=query, size=topk+1)
            x = res["hits"]["hits"]
            context = []
            for docu in x:
                cont = preprocess(docu["_source"]["text"])
                # groud truth 제거
                if cont == cp["original_context"]: continue
                context.append(preprocess(docu["_source"]["text"]))
            context = context[:topk]
            cp["context"] = "///".join(context)
            datas.append(cp)

        return pd.DataFrame(datas)

In [2]:
from datasets import load_from_disk

dataset = load_from_disk("../data/train_dataset")
train_datasets = dataset["train"]
valid_datasets = dataset["validation"]


In [3]:
x = elastic("wikipedia")
x.build_elatic()


<ipython-input-1-6d0488a02ff3>:61: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if self.es.indices.exists(self.index_name):
/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [4]:
p = x.retrieve_false(train_datasets, 100)


100%|██████████| 3952/3952 [04:58<00:00, 13.26it/s]


In [9]:
p.to_csv("/opt/ml/data/train_elastic_top100_noanswer.csv")

In [12]:
val = x.retrieve_false(valid_datasets, 100)
val.to_csv("/opt/ml/data/valid_elastic_top100_noanswer.csv")

  0%|          | 0/240 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
100%|██████████| 240/240 [00:18<00:00, 12.77it/s]


In [12]:
p.context[0]

'내각(內閣, cabinet)은 행정부의 주요 각료들로 구성되는 국가의 주요기관이다. 의원내각제에서 내각은 수상과 여러 장관으로 조직되는 합의체로, 국가의 행정권을 담당하고 국회에 대한 연대책임을 갖는다. 의원내각제에 있어서 내각은 국가행정의 최고기관인 한편 국민이 구성시키는 의회에 의하여 철저히 견제되어 의회민주주의 체제를 이룬다. 그 직접적 유래는 영국에서 국왕의 정치를 자문하던 추밀원에서 찾을 수 있다. 특히 내각은 추밀원의 일개 회의에서 시작하였다가 권한이 집중되어 분리된 기관으로, 이후 국왕의 실권이 사라지고 일명 웨스트민스터 시스템으로 불리는 의원내각제가 성립하면서 의회에 의한 민주적 행정부를 이루게 되었다. 한편 국가원수에게 대부분의 권력이 집중되는 대통령중심제와 군주제에서 내각은 원칙적으로 의결권이 없거나 의결의 구속력이 없는 보좌기관에 불과한 경우가 많다.(예: 대한민국의 국무회의) 대한민국은 국무회의가 내각에 속하며 권한이 대통령에 비해 제한적이다. 과거 왕조시대 때는 고려시대의 중서문하성, 중추원, 육부 또는 조선시대의 의정부와 육조가 내각과 비슷한 성향을 지니고 있었다.///이하의 국가들은 의회에서 행정부 수반을 선출한다는 점에서 내각제와 닮았다. 그리고 이들 국가들 중 상당수는 행정부 수반이 의회해산권을 갖고, 의회는 내각불신임권을 갖는데 이점 역시 내각제와 닮았다. 하지만 내각제와 달리 의회에서 선출되는 자는 행정부 수반의 지위 뿐만 아니라 국가 원수의 지위도 가진다. 그래서 의회에서 선출되는 자의 직위는 총리가 아니라, 대통령이다. 이처럼 1인이 행정부 수반과 국가 원수의 지위를 겸한다는 점에서 대통령중심제와 닮았다. 이상과 같은 이유로 이들 국가들의 정부 형태는 대통령제와 내각제의 절충형이지만, 행정 권한이 2인에게로 분리되어 있지 않으므로 이원집정부제는 아니다. * 나우루 * 남아프리카 공화국 * 마셜 제도 * 마카오 * 미얀마 * 미크로네시아 연방 * 보츠와나 * 산마리노 * 수리남 * 키리바시 * 홍콩///이하의 국가들은 의회

In [8]:
preprocess(train_datasets[0]['context']) in p.context[0]

False

In [13]:
preprocess(train_datasets[0]['context'])

'미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다. 미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다. 미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05'